In [71]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import datasets
import random 
import numpy.random as rand
from random import randrange
from scipy.stats import bernoulli, binom
from sklearn.linear_model import LinearRegression

In [3]:
# Set sample size N
N = 5000 

x = [1,0]

random.seed(560)

# Generate random variables 

treat_rand = np.random.choice(x, size = N,replace =True,p=[0.55,0.45])
comply_rand = np.random.choice(x, size = N,replace =True,p=[0.2,0.8])
age_rand = random.choices(list(range(18,61)),k = N)
post_rand = [0]*N
female_rand = np.random.binomial(1,0.51,N)
educ_rand = random.choices(list(range(3,8)),k = N)
a_rand =np.random.binomial (1,0.51,N)

# Create dictionaries for each of the generated variables

treat = {'treat':treat_rand}
comply = {'comply':comply_rand}
age = {'age':age_rand}
female = {'female':female_rand}
educ = {'educ':educ_rand}
a_dummy = {'a_dummy':a_rand}
post = {'post':post_rand}

# combine dictionaires and create dataframe
ds = {**treat,**comply,**educ,**age,**female,**a_dummy,**post}
df = pd.DataFrame(ds)

# Add id variable to the dataset 
df['id'] = range(1, len(df) + 1)

df2 = df
df2['post'] = 1

#append the two dataframes together
df3 = df.append(df2)


In [186]:
import pickle 

dataset_number = 8
seed = 0

name = 'dataset' + str(dataset_number)
with open('/Users/mawuliagamah/gitprojects/causal_inference/dataset1.pkl', 'rb') as f:
    save_dict = pickle.load(f)

In [187]:
y_obser = save_dict['y_obser']
x_obser = save_dict['x_obser']
y_inter = save_dict['y_inter']
x_inter = save_dict['x_inter']

In [189]:
print(y_obser)

[[-0.00430083]
 [ 0.45607497]
 [ 0.16422963]
 ...
 [ 0.3354805 ]
 [-0.30142274]
 [ 1.22610283]]


# 3. Generate simulation study 

## 3.1. RCT simulation

In [2]:
# Observations
n = 100

In [42]:
alpha = [1,2]
beta = [1,2]
gamma = [0.5,0.75]

# Anonymous function , ft , which takes arguments t and x and returns the expression on the rhs
ft = lambda t,x : alpha[t] + beta[t]*x + gamma[t]*x*x

In [107]:
# Generate variables (Measured covariates, unmeasured covariates , binary treatment and assignment).

d_rct = pd.DataFrame()

# Pre-treatment (Measured covariate)

d_rct['X1'] = np.random.uniform(-1,1 , size = (n,))
d_rct['X2'] = np.random.uniform(-1,1 , size = (n,))
d_rct['X3'] = np.random.uniform(-1,1 , size = (n,))
d_rct['X4'] = np.random.uniform(-1,1 , size = (n,))
d_rct['X5'] = np.random.uniform(-1,1 , size = (n,))

# Binary treatment assignment (drawn from bernoulli dsitribution with p = 0.5 )

p_bernoulli = 0.5
T_unform = np.random.rand(n) #treatment from uniform distribution

d_rct['T'] = (T_unform <= p_bernoulli).astype(np.int) # transform to bernoulli

# Error term 

epsilon_rct = np.random.normal(0,1 , size = (n,))

# Outcome 

het = 1 + d_rct['X1'] + d_rct['X1']**2 + d_rct['X2'] + d_rct['X2']**2 # Heterogenity of treatment effect
d_rct['Y'] = d_rct['T']*het+ d_rct.sum(axis=1) + epsilon_rct 




In [108]:
d_rct

,X1,X2,X3,X4,X5,T,Y
0,-0.968742,-0.460807,0.505358,-0.828222,-0.075093,0,-0.636370
1,-0.965901,-0.767507,-0.980156,-0.945298,-0.534608,1,-2.415667
2,-0.568051,0.010218,0.282352,-0.419805,-0.571435,0,-2.263836
3,0.617602,0.981857,-0.936727,-0.753496,-0.138203,0,-0.862383
4,-0.122196,-0.701707,0.553981,-0.675753,0.923602,1,0.639057
...,...,...,...,...,...,...,...
95,-0.799307,0.033335,-0.585390,0.339305,0.916316,0,-0.139965
96,0.211848,0.524060,0.246896,0.797433,0.437343,0,2.407272
97,0.744672,-0.582554,0.357081,-0.588045,0.151745,1,3.493979
98,-0.519987,0.166340,0.527461,-0.013476,-0.936616,1,0.796281
